In [1]:
#References
#https://github.com/sujitpal/holiday-similarity/blob/master/src/02-holidays-siamese-network.ipynb
#http://sujitpal.blogspot.com/2017/04/predicting-image-similarity-using.html
#https://medium.com/@ensembledme/writing-custom-keras-generators-fe815d992c5a
#https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
#https://github.com/ywpkwon/siamese_tf_mnist

In [2]:
from __future__ import division, print_function
from keras import backend as K
from keras.applications import vgg16
from keras.layers import Input, merge
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Dense, Dropout, Flatten, Lambda
from keras.models import Sequential, Model
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from random import shuffle
from scipy.misc import imresize
import itertools
import matplotlib.pyplot as plt
import numpy as np
from keras import optimizers
import os
%matplotlib inline

Using TensorFlow backend.


In [3]:
def load_image(image_name,img_dir):
    if image_name not in image_cache:
        image = plt.imread(os.path.join(img_dir, image_name)).astype(np.float32)
        image = imresize(image, (64, 64))
        image = np.divide(image, 255)
        image_cache[image_name] = image
    return image_cache[image_name]

In [4]:
def create_triples(image_dir):
    img_groups = {}
    for img_file in os.listdir(image_dir):
        prefix, suffix = img_file.split(".")
        gid, pid = prefix[0:4], prefix[4:]
        if gid in img_groups:
            img_groups[gid].append(pid)
        else:
            img_groups[gid] = [pid]
    pos_triples, neg_triples = [], []
    for key in img_groups.keys():
        triples = [(key + x[0] + ".png", key + x[1] + ".png", 1) 
                 for x in itertools.combinations(img_groups[key], 2)]
        pos_triples.extend(triples)
    group_names = list(img_groups.keys())
    for i in range(len(pos_triples)):
        g1, g2 = np.random.choice(np.arange(len(group_names)), size=2, replace=False)
        left = get_random_image(img_groups, group_names, g1)
        right = get_random_image(img_groups, group_names, g2)
        neg_triples.append((left, right, 0))
    shuffle(pos_triples)
    shuffle(neg_triples)
    return pos_triples

In [5]:
def get_random_image(img_groups, group_names, gid):
    gname = group_names[gid]
    photos = img_groups[gname]
    pid = np.random.choice(np.arange(len(photos)), size=1)[0]
    pname = photos[pid]
    return gname + pname + ".png"

In [6]:
def generate_image_triples_batch_training(image_triples, batch_size,no,shuffle=False):
    dirL=""
    if(no==1):
        dirL=seen_images_training_dir
    elif(no==2):
        dirL=unseen_images_training_dir
    else:
        dirL=shuffle_images_training_dir
    while True:
        if shuffle:
            indices = np.random.permutation(np.arange(len(image_triples)))
        else:
            indices = np.arange(len(image_triples))
        shuffled_triples = [image_triples[ix] for ix in indices]
        num_batches = len(shuffled_triples) // batch_size
        for bid in range(num_batches):
            # loop once per batch
            images_left, images_right, labels = [], [], []
            batch = shuffled_triples[bid * batch_size : (bid + 1) * batch_size]
            for i in range(batch_size):
                lhs, rhs, label = batch[i]
                images_left.append(load_image(lhs,dirL))
                images_right.append(load_image(rhs,dirL))              
                labels.append(label)
            Xlhs = np.array(images_left)
            Xrhs = np.array(images_right)
            Y = np_utils.to_categorical(np.array(labels), num_classes=2)
            yield ([Xlhs, Xrhs], Y)

            
BATCH_SIZE = 64

In [7]:
def generate_image_triples_batch_validation(image_triples, batch_size,no, shuffle=False):
    dirL=""
    if(no==1):
        dirL=seen_images_validation_dir
    elif(no==2):
        dirL=unseen_images_validation_dir
    else:
        dirL=shuffle_images_validation_dir

    while True:
        # loop once per epoch
        if shuffle:
            indices = np.random.permutation(np.arange(len(image_triples)))
        else:
            indices = np.arange(len(image_triples))
        shuffled_triples = [image_triples[ix] for ix in indices]
        num_batches = len(shuffled_triples) // batch_size
        for bid in range(num_batches):
            # loop once per batch
            images_left, images_right, labels = [], [], []
            batch = shuffled_triples[bid * batch_size : (bid + 1) * batch_size]
            for i in range(batch_size):
                lhs, rhs, label = batch[i]
                images_left.append(load_image(lhs,dirL))
                images_right.append(load_image(rhs,dirL))              
                labels.append(label)
            Xlhs = np.array(images_left)
            Xrhs = np.array(images_right)
            Y = np_utils.to_categorical(np.array(labels), num_classes=2)
            yield ([Xlhs, Xrhs], Y)

            
BATCH_SIZE = 64

In [8]:
def create_base_network(input_shape):
    seq = Sequential()
    seq.add(Conv2D(32,kernel_size=(3, 3),activation='relu',input_shape=input_shape,padding='valid'))
    seq.add(MaxPooling2D((2,2), padding='valid'))
    seq.add(Conv2D(64, (3, 3), activation='relu',padding='valid'))
    seq.add(MaxPooling2D((2,2),padding='valid'))
    seq.add(Conv2D(128, (3, 3), activation='relu',padding='valid'))
    seq.add(MaxPooling2D((2,2),padding='valid'))
    seq.add(Conv2D(256, (1, 1), activation='relu',padding='valid'))
    seq.add(MaxPooling2D((2,2),padding='valid'))
    seq.add(Conv2D(64, (1, 1), activation='relu',padding='valid'))
    seq.add(Flatten())
    seq.add(Dense(500))
    return seq


def cosine_distance(vecs, normalize=False):
    x, y = vecs
    if normalize:
        x = K.l2_normalize(x, axis=0)
        y = K.l2_normalize(x, axis=0)
    return K.prod(K.stack([x, y], axis=1), axis=1)

def cosine_distance_output_shape(shapes):
    return shapes[0]

def compute_accuracy(preds, labels):
    return labels[preds.ravel() < 0.5].mean()

In [9]:
# Directories For Seen,Unseen and Shuffled data
data_seen_dir = ("C:/Users/abhis/Desktop/us/Courses/AML 674/P2/seen-dataset/seen-dataset/")
data_unseen_dir = ("C:/Users/abhis/Desktop/us/Courses/AML 674/P2/unseen-dataset/unseen-dataset/")
data_shuffled_dir = ("C:/Users/abhis/Desktop/us/Courses/AML 674/P2/shuffled-dataset/shuffled-dataset/")

seen_images_training_dir = os.path.join(data_seen_dir, "TrainingSet/")
seen_images_validation_dir = os.path.join(data_seen_dir, "ValidationSet/")

unseen_images_training_dir = os.path.join(data_unseen_dir, "TrainingSet/")
unseen_images_validation_dir = os.path.join(data_unseen_dir, "ValidationSet/")

shuffle_images_training_dir = os.path.join(data_shuffled_dir, "TrainingSet/")
shuffle_images_validation_dir = os.path.join(data_shuffled_dir, "ValidationSet/")

In [10]:
#creating random triples from seen,unseen and shuffled
seen_image_pairs_training = create_triples(seen_images_training_dir)
seen_image_pairs_validation = create_triples(seen_images_validation_dir)

unseen_image_pairs_training = create_triples(unseen_images_training_dir)
unseen_image_pairs_validation = create_triples(unseen_images_validation_dir)

shuffle_image_pairs_training = create_triples(shuffle_images_training_dir)
shuffle_image_pairs_validation = create_triples(shuffle_images_validation_dir)

In [11]:
#base for siamese network
input_shape = (64, 64, 3)
base_network = create_base_network(input_shape)

image_left = Input(shape=input_shape)
image_right = Input(shape=input_shape)

vector_left = base_network(image_left)
vector_right = base_network(image_right)

distance = Lambda(cosine_distance, output_shape=cosine_distance_output_shape)([vector_left, vector_right])

fc1 = Dense(128, kernel_initializer="glorot_uniform")(distance)
fc1 = Dropout(0.2)(fc1)
fc1 = Activation("relu")(fc1)

pred = Dense(2, kernel_initializer="glorot_uniform")(fc1)
pred = Activation("softmax")(pred)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
#creating model 1
model = Model(inputs=[image_left, image_right], outputs=pred)
adam = optimizers.Adam(lr=0.00001)
model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])

In [ ]:
#running epochs for seen data
NUM_EPOCHS = 10 
image_cache = {}
train_gen = generate_image_triples_batch_training(seen_image_pairs_training, BATCH_SIZE,1, shuffle=True)
val_gen = generate_image_triples_batch_validation(seen_image_pairs_validation, BATCH_SIZE,1, shuffle=False)

num_train_steps = len(seen_image_pairs_training) // BATCH_SIZE
num_val_steps = len(seen_image_pairs_validation) // BATCH_SIZE

history = model.fit_generator(train_gen,
                              steps_per_epoch=num_train_steps,
                              epochs=NUM_EPOCHS,
                              validation_data=val_gen,
                              validation_steps=num_val_steps)

In [14]:
#creating model 2
model2 = Model(inputs=[image_left, image_right], outputs=pred)
adam = optimizers.Adam(lr=0.00001)
model2.compile(loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])

In [ ]:
#running epochs for UnSeen image
NUM_EPOCHS = 15
image_cache = {}
train_gen = generate_image_triples_batch_training(unseen_image_pairs_training, BATCH_SIZE,2, shuffle=True)
val_gen = generate_image_triples_batch_validation(unseen_image_pairs_validation, BATCH_SIZE,2, shuffle=False)

num_train_steps = len(unseen_image_pairs_training) // BATCH_SIZE
num_val_steps = len(unseen_image_pairs_validation) // BATCH_SIZE

history = model2.fit_generator(train_gen,
                              steps_per_epoch=num_train_steps,
                              epochs=NUM_EPOCHS,
                              validation_data=val_gen,
                              validation_steps=num_val_steps)

In [16]:
#creating model 3
model3 = Model(inputs=[image_left, image_right], outputs=pred)
adam = optimizers.Adam(lr=0.00001)
model3.compile(loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])

In [ ]:
#running epochs for Shuffled image
NUM_EPOCHS = 10 
image_cache = {}
train_gen = generate_image_triples_batch_training(shuffle_image_pairs_training, BATCH_SIZE,3, shuffle=True)
val_gen = generate_image_triples_batch_validation(shuffle_image_pairs_validation, BATCH_SIZE,3, shuffle=False)

num_train_steps = len(shuffle_image_pairs_training) // BATCH_SIZE
num_val_steps = len(shuffle_image_pairs_validation) // BATCH_SIZE

history = model3.fit_generator(train_gen,
                              steps_per_epoch=num_train_steps,
                              epochs=NUM_EPOCHS,
                              validation_data=val_gen,
                              validation_steps=num_val_steps)